In [1]:
# Librerias necesarias:

from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as ec
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
import yfinance as yf
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
# Modulo iniciar driver personalizado con algunos ajustes y mi user agent

def INICIAR_DRIVER():
    options=Options()
    userAgent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15'
    options.add_argument(f'user-agent={userAgent}')
    options.add_argument('--start-maximized')
    options.add_argument('--disable-web-security')
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-notifications')
    options.add_argument('--ignore-certificate-errors')
    # QUITA EL HEADLESS SI QUIERES QUE SE VEA LO QUE OCURRE EN CHROME
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument('--no-default-browser-check')
    options.add_argument('--no-first-run')
    options.add_argument('--no-proxy-server')
    options.add_argument('--disable-blink-features=AutomationControlled')
    exp_opt=['enable-automation', 'ignore-certificate-errors','enable-logging']
    options.add_experimental_option('excludeSwitches', exp_opt)
    prefs={'profile.default_content_setting_values.notifications':2, 'intl.accept_languages':['es-ES','es'],'credentials_enable_service':False}
    options.add_experimental_option('prefs',prefs)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    return driver


In [3]:
# Modulo abrir driver, quitar la notificación del inicio y guardar la sopa de cada página

def entrar_driver():
    print('Empezando programa')
    print()
    driver=INICIAR_DRIVER()
    wait=WebDriverWait(driver,1.5)
    path='https://stockanalysis.com/stocks'
    driver.get(path)
    # La notificación tarda 24 segundos aproximadamente en salir
    time.sleep(25)
    boton = driver.find_element(By.XPATH, "/html/body/div/div[2]/div/button")
    print('Saltó la notificación')
    print()
    time.sleep(1)
    boton.click()
    print('Se bloqueo la notificación')
    print()
    time.sleep(1)
    print('Recogiendo las soups')
    print()
    time.sleep(1)
    lista=[]
    for i in range (12):
        time.sleep(1.5)
        res = driver.page_source
        soup = BeautifulSoup(res,'html.parser')
        lista.append(soup)
        if i!=11:
            siguiente_bt = wait.until(ec.element_to_be_clickable((By.XPATH,'//*[@id="main"]/div/div/nav/button[2]')))
            siguiente_bt.click()
    return lista

In [4]:
# Dada la url de la empresa saca sus datos

def valores(url,dic_og):
    ress =requests.get(url,headers=headers)
    sopa=BeautifulSoup(ress.text,'html.parser')
    tabla = sopa.find('table',class_='w-full border-separate border-spacing-0 whitespace-nowrap')

    lista_Y=[]
    fila_anyos=tabla.find('thead')
    anyos=fila_anyos.find_all('th')

    todas = tabla.find('tbody')
    variables = todas.find_all('tr')

    for i in range(1,len(anyos)):
        anyo=anyos[i]
        anyo=anyo.text 
        if anyo != 'None' and '+' not in list(set(anyo)):
            dic={}
            dic = dict(dic_og)
            dic['Date']= anyo
            for variable in variables:
                lista_valores = variable.find_all('td')
                nombre_variable = lista_valores[0].text.strip()
                valor = lista_valores[i].text
                dic[nombre_variable]=valor
                
            lista_Y.append(dic)
            
    return lista_Y

In [5]:
# Dada la url de la empresa saca sus datos de ratio (las etiquetas html son distintas)

def valores_ratio(url,dic_og):
    ress =requests.get(url,headers=headers)
    sopa=BeautifulSoup(ress.text,'html.parser')
    tabla = sopa.find('table',class_='w-full border-separate border-spacing-0 whitespace-nowrap')

    lista_Y=[]
    
    fila_anyos=tabla.find('thead')
    anyos=fila_anyos.find_all('th')

    todas = tabla.find('tbody')
    variables = todas.find_all('tr')

    for i in range(2,len(anyos)-1):
        anyo=anyos[i]
        anyo=anyo.text
        if anyo != 'None' and '+' not in list(set(anyo)):
            dic = dict(dic_og)
            dic['Date']= anyo
            for variable in variables:
                lista_valores = variable.find_all('td')
                nombre_variable = lista_valores[0].text.strip()
                valor = lista_valores[i].text
                dic[nombre_variable]=valor

            lista_Y.append(dic)
            
    return lista_Y

In [6]:
# Crea una lista de diccionarios con los datos de todas las fechas de la empresa dada

def empresa(tupla):
    
    url_adapts=['financials/?p=quarterly','financials/balance-sheet/?p=quarterly',
        'financials/cash-flow-statement/?p=quarterly','financials/ratios/?p=quarterly']

    url_og='https://stockanalysis.com'

    dic_og = {}
    dic_og['Ticker'] = tupla[1]
    dic_og['Name'] = tupla[2]
    dic_og['Sector'] = tupla[3]
    
    lista = []
    for i in range(len(url_adapts)):

        if i == 3:
            url = url_og + tupla[0] + url_adapts[i]
            lista.append(valores_ratio(url,dic_og))
        else:
            url = url_og + tupla[0] + url_adapts[i]
            lista.append(valores(url,dic_og))

    dics = []
    
    for i in range(len(lista[0])):
        dic = lista[0][i]
        dic.update(lista[1][i])
        dic.update(lista[2][i])
        dic.update(lista[3][i])
        dics.append(dic)

    return dics

            
    

In [7]:
# Actualiza el data frame con los datos de las empresas

def update_df(dics,df):
    df_temp = pd.DataFrame(dics)
    df = pd.concat([df,df_temp], ignore_index=True)
    return df
    

In [8]:
# Modulo principal, va recorriendo las empresas y sacando los datos

def get_data():
    lista_soups = entrar_driver()
    df = pd.DataFrame()
    for i in lista_soups:
        tabla = i.find('div',class_='table-wrap svelte-172ru7t')
        tabla_tb = tabla.find('tbody')
        tk_list = tabla_tb.find_all('tr')
        for j in tk_list:
            time.sleep(0.01)
            try:
                link=j.a['href']
                ticker = j.a.text
                nombre = j.find_all('td')[1].text
                sector = j.find_all('td')[2].text
                tupla = (link, ticker, nombre, sector)
                dics = empresa(tupla)
                df = update_df(dics,df)
                print('HECHA: ', ticker)
            except:
                print('ERROR: ', ticker)
    return df

In [9]:
# Añadirle 3 meses a la fecha

def dates(d2):
    
    obj_date = datetime.strptime(d2, "%Y-%m-%d")
    new_date = obj_date + relativedelta(months=3)

    return new_date.strftime("%Y-%m-%d")

In [10]:
# Obtener el precio de cierre dada una fecha y la data

def get_close(d, data):
    try:
        d = pd.Timestamp(d)
        
        if d in data.index:
            c = data.loc[d, 'Close']
        else:
            dif = np.abs(data.index - d)
            closest = dif.argmin()
            date = data.index[closest]
            dias = dif[closest].days
            if dias <= 7:
                c = data.loc[date, 'Close']
            else:
                c = '-'
    except:
        c = '-'
    return c

In [11]:
# Recorrer el dataframe

def travel_data(data,ini,end,df):
    for i in range(ini,end+1):
        d1 = df.iloc[i,3]
        d2 = dates(d1)
        c1 = get_close(d1,data)
        c2 = get_close(d2,data)
        df.loc[i, 'Price M1'] = c1
        df.loc[i, 'Price M2'] = c2
    return df

In [12]:
# Añadir precio de sus acciones en bolsa, en la fecha del scraping y a las 3 meses siguientes

def close_values(df):
    ini = 0
    df['Price M1'] = pd.NA
    df['Price M2'] = pd.NA

    for i in range(len(df)-1):
        if df.iloc[i,0] != df.iloc[i+1,0]:
            end = i
            d2 = df.iloc[ini,3]
            date1 = df.iloc[end,3]

            date2 = dates(d2)
            tk = df.iloc[ini,0]
            
            data = yf.download(tk, start=date1, end=date2)
            
            df = travel_data(data, ini, end, df) 

            ini = i + 1

    return df

In [13]:
# Sustituye los datos faltantes por datos nulos tipo pandas

def clean_data(df):
    
    df.replace('-', pd.NA, inplace=True)
    df.replace('Upgrade\n\t\t', pd.NA, inplace=True)
    
    df = df.replace({'%': ''}, regex=True)
    df = df.replace({',': ''}, regex=True)
    
    return df

In [14]:
# Programa principal

def scraper():

    df = get_data()
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].astype(str)
    df = close_values(df)
    df = clean_data(df)
    df.to_csv('STOCK_DATA.csv')

In [ ]:
# EJECUTAR EL PROGRAMA 

# Estos son mis headers 
headers={'user_agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15'}

scraper()

Empezando programa

Saltó la notificación

Se bloqueo la notificación

Recogiendo las soups

